# 网络构建

神经网络模型是由神经网络层和Tensor操作构成的，[mindspore.nn](https://www.mindspore.cn/docs/zh-CN/r2.4.10/api_python/mindspore.nn.html)提供了常见神经网络层的实现，在MindSpore中，[Cell](https://www.mindspore.cn/docs/zh-CN/r2.4.10/api_python/nn/mindspore.nn.Cell.html)类是构建所有网络的基类，也是网络的基本单元。一个神经网络模型表示为一个`Cell`，它由不同的子`Cell`构成。使用这样的嵌套结构，可以简单地使用面向对象编程的思维，对神经网络结构进行构建和管理。

下面我们将构建一个用于Mnist数据集分类的神经网络模型。

In [1]:
%%capture captured_output
# 实验环境已经预装了mindspore==2.4.10，如需更换mindspore版本，可更改下面 MINDSPORE_VERSION 变量
!pip uninstall mindspore -y
%env MINDSPORE_VERSION=2.4.10
!pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/${MINDSPORE_VERSION}/MindSpore/unified/aarch64/mindspore-${MINDSPORE_VERSION}-cp39-cp39-linux_aarch64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple

In [2]:
# 查看当前 mindspore 版本
!pip show mindspore

Name: mindspore
Version: 2.4.10
Summary: MindSpore is a new open source deep learning training/inference framework that could be used for mobile, edge and cloud scenarios.
Home-page: https://www.mindspore.cn
Author: The MindSpore Authors
Author-email: contact@mindspore.cn
License: Apache 2.0
Location: /home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages
Requires: asttokens, astunparse, numpy, packaging, pillow, protobuf, psutil, safetensors, scipy
Required-by: 


In [3]:
import mindspore
from mindspore import nn, ops

/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


## 定义模型类

当我们定义神经网络时，可以继承`nn.Cell`类，在`__init__`方法中进行子Cell的实例化和状态管理，在`construct`方法中实现Tensor操作。

> `construct`意为神经网络（计算图）构建，相关内容详见[使用静态图加速](https://www.mindspore.cn/tutorials/zh-CN/r2.4.10/beginner/accelerate_with_static_graph.html)。

In [4]:
class Network(nn.Cell):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.dense_relu_sequential = nn.SequentialCell(
            nn.Dense(28*28, 512, weight_init="normal", bias_init="zeros"),
            nn.ReLU(),
            nn.Dense(512, 512, weight_init="normal", bias_init="zeros"),
            nn.ReLU(),
            nn.Dense(512, 10, weight_init="normal", bias_init="zeros")
        )

    def construct(self, x):
        x = self.flatten(x)
        logits = self.dense_relu_sequential(x)
        return logits

构建完成后，实例化`Network`对象，并查看其结构。

In [5]:
model = Network()
print(model)

Network<
  (flatten): Flatten<>
  (dense_relu_sequential): SequentialCell<
    (0): Dense<input_channels=784, output_channels=512, has_bias=True>
    (1): ReLU<>
    (2): Dense<input_channels=512, output_channels=512, has_bias=True>
    (3): ReLU<>
    (4): Dense<input_channels=512, output_channels=10, has_bias=True>
    >
  >


我们构造一个输入数据，直接调用模型，可以获得一个二维的Tensor输出，其包含每个类别的原始预测值。

> `model.construct()`方法不可直接调用。

In [6]:
X = ops.ones((1, 28, 28), mindspore.float32)
logits = model(X)
# print logits
logits

Tensor(shape=[1, 10], dtype=Float32, value=
[[ 5.54037513e-04, -1.02895824e-03,  3.69491475e-03 ...  8.76324996e-03, -4.03808383e-03,  6.91894093e-04]])

在此基础上，我们通过一个`nn.Softmax`层实例来获得预测概率。

In [7]:
pred_probab = nn.Softmax(axis=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: [7]


## 模型层

本节中我们分解上节构造的神经网络模型中的每一层。首先我们构造一个shape为(3, 28, 28)的随机数据（3个28x28的图像），依次通过每一个神经网络层来观察其效果。

In [8]:
input_image = ops.ones((3, 28, 28), mindspore.float32)
print(input_image.shape)

(3, 28, 28)


### nn.Flatten

实例化[nn.Flatten](https://www.mindspore.cn/docs/zh-CN/r2.4.10/api_python/nn/mindspore.nn.Flatten.html)层，将28x28的2D张量转换为784大小的连续数组。

In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.shape)

(3, 784)


### nn.Dense

[nn.Dense](https://www.mindspore.cn/docs/zh-CN/r2.4.10/api_python/nn/mindspore.nn.Dense.html)为全连接层，其使用权重和偏差对输入进行线性变换。

In [10]:
layer1 = nn.Dense(in_channels=28*28, out_channels=20)
hidden1 = layer1(flat_image)
print(hidden1.shape)

(3, 20)


### nn.ReLU

[nn.ReLU](https://www.mindspore.cn/docs/zh-CN/r2.4.10/api_python/nn/mindspore.nn.ReLU.html)层给网络中加入非线性的激活函数，帮助神经网络学习各种复杂的特征。

In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: [[-0.63215446 -0.75162077  1.2194451  -1.1919035   0.6814073   0.10918756
   0.13706607 -0.19950262  0.2533091   0.8950758  -0.22780897  0.3421619
  -0.20590624  0.699736   -0.54346323 -0.84042156  0.32450992 -0.18706635
  -0.672404   -0.20169368]
 [-0.63215446 -0.75162077  1.2194451  -1.1919035   0.6814073   0.10918756
   0.13706607 -0.19950262  0.2533091   0.8950758  -0.22780897  0.3421619
  -0.20590624  0.699736   -0.54346323 -0.84042156  0.32450992 -0.18706635
  -0.672404   -0.20169368]
 [-0.63215446 -0.75162077  1.2194451  -1.1919035   0.6814073   0.10918756
   0.13706607 -0.19950262  0.2533091   0.8950758  -0.22780897  0.3421619
  -0.20590624  0.699736   -0.54346323 -0.84042156  0.32450992 -0.18706635
  -0.672404   -0.20169368]]


After ReLU: [[0.         0.         1.2194451  0.         0.6814073  0.10918756
  0.13706607 0.         0.2533091  0.8950758  0.         0.3421619
  0.         0.699736   0.         0.         0.32450992 0.
  0.         0.        ]
 [0.    

### nn.SequentialCell

[nn.SequentialCell](https://www.mindspore.cn/docs/zh-CN/r2.4.10/api_python/nn/mindspore.nn.SequentialCell.html)是一个有序的Cell容器。输入Tensor将按照定义的顺序通过所有Cell。我们可以使用`nn.SequentialCell`来快速组合构造一个神经网络模型。

In [12]:
seq_modules = nn.SequentialCell(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Dense(20, 10)
)

logits = seq_modules(input_image)
print(logits.shape)

(3, 10)


### nn.Softmax

最后使用[nn.Softmax](https://www.mindspore.cn/docs/zh-CN/r2.4.10/api_python/nn/mindspore.nn.Softmax.html)将神经网络最后一个全连接层返回的logits的值缩放为\[0, 1\]，表示每个类别的预测概率。`axis`指定的维度数值和为1。

In [13]:
softmax = nn.Softmax(axis=1)
pred_probab = softmax(logits)

## 模型参数

网络内部神经网络层具有权重参数和偏置参数（如`nn.Dense`），这些参数会在训练过程中不断进行优化，可通过 `model.parameters_and_names()` 来获取参数名及对应的参数详情。

In [14]:
print(f"Model structure: {model}\n\n")

for name, param in model.parameters_and_names():
    print(f"Layer: {name}\nSize: {param.shape}\nValues : {param[:2]} \n")

Model structure: Network<
  (flatten): Flatten<>
  (dense_relu_sequential): SequentialCell<
    (0): Dense<input_channels=784, output_channels=512, has_bias=True>
    (1): ReLU<>
    (2): Dense<input_channels=512, output_channels=512, has_bias=True>
    (3): ReLU<>
    (4): Dense<input_channels=512, output_channels=10, has_bias=True>
    >
  >


Layer: dense_relu_sequential.0.weight
Size: (512, 784)
Values : [[ 1.3819782e-04  5.0520324e-03  4.6783945e-05 ...  2.4931242e-03
  -9.8474715e-03  1.2437404e-03]
 [-1.1515701e-02 -4.3822494e-03  1.7790269e-02 ...  8.7975888e-03
   1.5508163e-03 -6.4374157e-03]] 

Layer: dense_relu_sequential.0.bias
Size: (512,)
Values : [0. 0.] 

Layer: dense_relu_sequential.2.weight
Size: (512, 512)
Values : [[ 0.01220912  0.01043043  0.00819144 ...  0.00323056 -0.01690722
   0.00669364]
 [ 0.0013445   0.00232303  0.01272476 ...  0.00236821 -0.00454598
  -0.01181476]] 

Layer: dense_relu_sequential.2.bias
Size: (512,)
Values : [0. 0.] 

Layer: dense_relu_sequ

更多内置神经网络层详见[mindspore.nn API](https://www.mindspore.cn/docs/zh-CN/r2.4.10/api_python/mindspore.nn.html)。